In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2022-03-26 02:16:23--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-03-26 02:16:24--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.02MB/s    in 2m 41s  

2022-03-26 02:19:04 (5.12 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

/bin/bash: !unzip: command not found


In [5]:
!unzip glove.6B.zip -d '/content/drive/MyDrive/Colab Notebooks/dataset'

Archive:  glove.6B.zip
  inflating: /content/drive/MyDrive/Colab Notebooks/dataset/glove.6B.50d.txt  
  inflating: /content/drive/MyDrive/Colab Notebooks/dataset/glove.6B.100d.txt  
  inflating: /content/drive/MyDrive/Colab Notebooks/dataset/glove.6B.200d.txt  
  inflating: /content/drive/MyDrive/Colab Notebooks/dataset/glove.6B.300d.txt  


In [6]:
!pip3 install gensim

In [7]:
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.decomposition import PCA

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [46]:
glove_file = datapath('/content/drive/MyDrive/Colab Notebooks/dataset/glove.6B.300d.txt')
word2vec_glove_file = get_tmpfile("glove.6B.300d.word2vec.txt")
glove2word2vec(glove_file, word2vec_glove_file)

(400000, 300)

In [47]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [48]:
model.most_similar('obama')

[('barack', 0.9254721403121948),
 ('mccain', 0.7590768337249756),
 ('bush', 0.7570987939834595),
 ('clinton', 0.7085603475570679),
 ('hillary', 0.6497915983200073),
 ('kerry', 0.6144052743911743),
 ('rodham', 0.6138635873794556),
 ('biden', 0.5940852165222168),
 ('gore', 0.5885976552963257),
 ('democrats', 0.5608304738998413)]

In [49]:
model.most_similar('banana')

[('bananas', 0.6691170930862427),
 ('mango', 0.580410361289978),
 ('pineapple', 0.5492371916770935),
 ('coconut', 0.5462779402732849),
 ('papaya', 0.541056752204895),
 ('fruit', 0.5218108296394348),
 ('growers', 0.4877638816833496),
 ('nut', 0.4839959144592285),
 ('peanut', 0.48062020540237427),
 ('potato', 0.4806118607521057)]

In [50]:
model.most_similar(negative='banana')

[('keyrates', 0.6847262382507324),
 ('rw97', 0.6595869064331055),
 ('+9.00', 0.6340475678443909),
 ('ryryryryryry', 0.6322759985923767),
 ('zety', 0.5784541368484497),
 ('.0342', 0.5776804089546204),
 ('k586-1', 0.5598777532577515),
 ('cw96', 0.5540916323661804),
 ('mongkolporn', 0.5488854050636292),
 ('purva.patel@chron.com', 0.5483731627464294)]

In [51]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.6713


In [52]:
def analogy(x1, x2, y1):
  result = model.most_similar(positive=[y1, x2], negative=[x1])
  return result[0][0]

In [53]:
analogy('japan', 'japanese', 'australia')

'australian'

In [54]:
analogy('australia', 'beer', 'france')

'champagne'

In [55]:
analogy('obama', 'clinton', 'reagan')

'ronald'

In [56]:
analogy('tall', 'tallest', 'long')

'longest'

In [57]:
analogy('good', 'fantastic', 'bad')

'horrible'

In [58]:
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

cereal


/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [59]:
def display_pca_scatterplot(model, words=None, sample=0):
  if words == None:
    if sample > 0:
      words = np.random.choice(list(model.vocab.keys()), sample)
    else:
      words = [word for word in model.vocab]
  
  words_vectors = np.array([model[w] for w in words])

  twodim = PCA().fit_transform(words_vectors)[:,:2]

  plt.figure(figsize=(6,6))
  plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
  for word, (x, y) in zip(words, twodim):
    plt.text(x+0.05, y+0.05, word)

In [60]:
display_pca_scatterplot(model, 
                        ['coffee', 'tea', 'beer', 'wine', 'brandy', 'rum', 'champagne', 'water',
                         'spaghetti', 'borscht', 'hamburger', 'pizza', 'falafel', 'sushi', 'meatballs',
                         'dog', 'horse', 'cat', 'monkey', 'parrot', 'koala', 'lizard',
                         'frog', 'toad', 'monkey', 'ape', 'kangaroo', 'wombat', 'wolf',
                         'france', 'germany', 'hungary', 'luxembourg', 'australia', 'fiji', 'china',
                         'homework', 'assignment', 'problem', 'exam', 'test', 'class',
                         'school', 'college', 'university', 'institute'])

<IPython.core.display.Javascript object>

In [61]:
display_pca_scatterplot(model, sample=300)

<IPython.core.display.Javascript object>